## Goal of this Notebook

In this notebook we use langchain to build a simple RAG to Ollama and we ask the llama3 model for weather reports from the weather context fed from Milvus.

### Simple Retrieval-Augmented Generation (RAG) with LangChain:

Build a simple Python [RAG](https://milvus.io/docs/integrate_with_langchain.md) application (mediumrag.py) to use Milvus for asking about the current weather via OLLAMA.   While outputing to the screen we also send the results to Slack formatted as Markdown.

### 🔍 Summary
By the end of this application, you’ll have a comprehensive understanding of using Milvus, data ingest object semi-structured and unstructured data, and using Open Source models to build a robust and efficient data retrieval system.  





In [10]:
import os
from pymilvus import MilvusClient
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama
from langchain import hub
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import requests
import sys
import io
import json
import shutil
import sys
import datetime
import subprocess
import math
import base64
from time import gmtime, strftime
import random, string
import time
import psutil
import base64
import uuid
import socket
from pymilvus import connections
from pymilvus import utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
import torch
from torchvision import transforms
from PIL import Image
import timm
from sklearn.preprocessing import normalize
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from IPython.display import display
from pymilvus import MilvusClient
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from dotenv import load_dotenv
load_dotenv(verbose=True)
import requests
from datetime import datetime, timedelta
from langchain_milvus import Milvus
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from langchain_core.globals import set_verbose, set_debug

# Disable verbose logging
set_verbose(False)

# Disable debug logging
set_debug(False)

### Setup environment and constants

DIMENSION = 384 
MILVUS_URL = "http://192.168.1.153:19530" 

slack_token = os.environ.get("SLACK_BOT_TOKEN")
client = WebClient(token=slack_token)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
COLLECTION_NAME = "mediumarticles"
DEFAULT_QUERY = "What is the current weather in detail from Central park."

### Environment Variables needed
os.environ["LANGCHAIN_HUB_API_URL"] = "https://api.hub.langchain.com"
os.environ["LANGCHAIN_HUB_API_KEY"] = os.environ.get("LANGCHAIN_HUB_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGCHAIN_HUB_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.hub.langchain.com"
os.environ["TOKENIZERS_PARALLELISM"]  = "true"

### Turn off slack warnings
os.environ["SKIP_SLACK_SDK_WARNING"] = "false"

# https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.milvus.Milvus.html 

/Users/timothyspann/Downloads/code/milvusvenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
### Set up connection to Milvus for medium articles
vector_store = Milvus(
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME,
    primary_field = "doc_id",
    vector_field = "doc_embedding",
    text_field="doc_text",
    connection_args={"uri": MILVUS_URL},
)

In [14]:
# https://zilliz.com/blog/building-multilingual-rag-milvus-langchain-openai
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

#### Run query
def run_query() -> None:
    llm = Ollama(
        model="llama3.1",
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        stop=["<|eot_id|>"],
    )

    query = input("\nQuery: ")
    qa_chain = RetrievalQA.from_chain_type(
        llm, retriever=vector_store.as_retriever(collection =  COLLECTION_NAME))

    result = qa_chain.invoke({"query": query})
    
    response = str(result["result"])
    # print(resultforslack)
    
    try:
        slackresponse =  str(response).replace("**","*")
                                   
        slackresponse = client.chat_postMessage(mrkdwn=True, channel="C06NE1FU6SE", text="", 
                                            blocks=[{"type": "section","text": {"type": "mrkdwn","text": str(slackresponse) +"\n" }}])
    except SlackApiError as e:
        print("Slack failed")   

In [ ]:
while True:
    run_query()

/var/folders/c2/6wq9klzn19d97gvw8f8j1jdr0000gn/T/ipykernel_20902/1639999201.py:8: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(



Query:  What is Milvus?


Milvus is a vector database designed for efficient storage and retrieval of vectors (multi-dimensional numerical arrays), making it suitable for various applications such as computer vision, natural language processing, and more. It provides fast and scalable data management capabilities, enabling users to store, query, and analyze large-scale vector datasets with ease.

/var/folders/c2/6wq9klzn19d97gvw8f8j1jdr0000gn/T/ipykernel_20902/1639999201.py:8: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(


In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template="""
   Use the context to answer questions about Milvus and format in Slack mrkdwn format.
   {search}
   Context: {context}
   Answer:"""
  
prompt = ChatPromptTemplate.from_template(template)

chain = (
      {"context": retriever, "search": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

query = input("Search: ")

response = chain.invoke(query)

try:
    slackresponse =  str(response).replace("**","*")
                               
    slackresponse = client.chat_postMessage(mrkdwn=True, channel="C06NE1FU6SE", text="", 
                                        blocks=[{"type": "section","text": {"type": "mrkdwn","text": str(slackresponse) +"\n" }}])
except SlackApiError as e:
    print("Slack failed")
        

NameError: name 'retriever' is not defined